In [1]:
import os
# Add these two lines to disable the GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import pandas as pd
from sqlalchemy import create_engine
import sklearn
import tensorflow as tf

# --- Configuration ---
# Database connection details (should be the same as in etl.py)
DB_USER = 'cosmic_admin'
DB_PASSWORD = 'Raghav' # IMPORTANT: Replace with your password
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'cosmic_db'
TABLE_NAME = 'celestial_objects'

# --- 1. Connect to the Database ---
engine_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(engine_string)

print("✅ Connected to PostgreSQL")
# Check if TensorFlow can see a GPU (it should say 0 now)
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Scikit-learn Version: {sklearn.__version__}")


# --- 2. Load a Sample of the Data ---
sample_size = 100000
sql_query = f"SELECT * FROM {TABLE_NAME} LIMIT {sample_size};"

print(f"\nLoading {sample_size} rows from the '{TABLE_NAME}' table...")
df = pd.read_sql(sql_query, engine)
print("✅ Data loaded successfully!")

# --- 3. Verify the Data ---
print("\nFirst 5 rows of the dataset:")
display(df.head())

print("\nDataset Information:")
df.info()

2025-09-17 11:22:17.672184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-17 11:22:17.704395: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-17 11:22:17.967012: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-17 11:22:18.186894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758108138.353964    7414 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758108138.40

✅ Connected to PostgreSQL
Num GPUs Available: 0
TensorFlow Version: 2.19.0
Scikit-learn Version: 1.6.1

Loading 100000 rows from the 'celestial_objects' table...


2025-09-17 11:22:22.637414: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Data loaded successfully!

First 5 rows of the dataset:


,main_id,otype,sp_type,rvz_nature,coo_wavelength,coo_bibcode,mesdistance.bibcode,mesdistance.method,mesdistance.unit,ra,...,plx_value,rvz_redshift,mesdistance.dist,mesdistance.dist_prec,mesdistance.mespos,mesdistance.minus_err,mesdistance.minus_err_prec,mesdistance.plus_err,mesdistance.plus_err_prec,mesdistance.qual
0,CPD-89 32,*,Unknown,s,O,2018yCat.1345....0G,2018yCat.1345....0G,paral,pc,126.170378,...,2.9562,0.000034,338.2721,4.0,1.0,-31.6737,4.0,31.6737,4.0,0.0
1,Gaia DR3 4611707527624020352,G,Unknown,Unknown,O,2020yCat.1350....0G,Unknown,Unknown,Unknown,33.661306,...,0.0000,0.000000,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0
2,Gaia DR3 4611710830456309376,G,Unknown,Unknown,O,2020yCat.1350....0G,Unknown,Unknown,Unknown,29.199414,...,0.0000,0.000000,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0
3,Gaia DR3 5764629066590886528,EB*,Unknown,Unknown,O,2020yCat.1350....0G,Unknown,Unknown,Unknown,214.695005,...,0.3848,0.000000,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0
4,Gaia DR3 6341072604664980864,PM*,Unknown,s,O,2020yCat.1350....0G,2020yCat.1350....0G,paral,pc,328.469785,...,11.5578,0.000081,86.5217,4.0,1.0,-0.1273,4.0,0.1273,4.0,0.0



Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 38 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   main_id                     100000 non-null  object 
 1   otype                       100000 non-null  object 
 2   sp_type                     100000 non-null  object 
 3   rvz_nature                  100000 non-null  object 
 4   coo_wavelength              100000 non-null  object 
 5   coo_bibcode                 100000 non-null  object 
 6   mesdistance.bibcode         100000 non-null  object 
 7   mesdistance.method          100000 non-null  object 
 8   mesdistance.unit            100000 non-null  object 
 9   ra                          100000 non-null  float64
 10  dec                         100000 non-null  float64
 11  coo_err_maj                 100000 non-null  float64
 12  coo_err_min                 100000 non-null  float6

In [2]:
from sklearn.preprocessing import StandardScaler

# --- 4. Feature Selection ---
# We need to separate the columns that are useful for training from those
# that are just identifiers or text-based categories.

# List of columns to drop before training the model
# We drop IDs, text descriptions, and any columns that are mostly empty or not useful.
# We also drop ra/dec for now as their raw values aren't as useful as their combination with other features.
columns_to_drop = [
    'main_id', 'otype', 'sp_type', 'rvz_nature', 'coo_wavelength',
    'coo_bibcode', 'mesdistance.bibcode', 'mesdistance.method', 'mesdistance.unit',
    'ra', 'dec' # We will revisit using coordinates later
]

# Create a new dataframe containing only the features we'll use for training
# We use .drop() and specify errors='ignore' in case some columns don't exist in the sample
numerical_df = df.drop(columns=columns_to_drop, errors='ignore')

# It's possible some text columns slipped through our ETL cleaning.
# Let's be extra safe and select only numeric types for our final training data.
numerical_df = numerical_df.select_dtypes(include=['number'])

print(f"Selected {len(numerical_df.columns)} features for training.")
print("Features:", numerical_df.columns.tolist())


# --- 5. Scaling the Data ---
# We use StandardScaler to normalize the features. It transforms the data
# such that its distribution will have a mean value of 0 and a standard deviation of 1.

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the data and transform it
# This calculates the mean and standard deviation for each column and applies the transformation.
X_scaled = scaler.fit_transform(numerical_df)

print("\n✅ Data has been scaled successfully.")
print("Shape of the scaled data:", X_scaled.shape)

# Display the first 5 rows of the scaled data to see the result
# Notice how the values are now centered around zero.
print("\nFirst 5 rows of scaled data:")
display(pd.DataFrame(X_scaled, columns=numerical_df.columns).head())

Selected 27 features for training.
Features: ['coo_err_maj', 'coo_err_min', 'coo_err_angle', 'J', 'V', 'U', 'H', 'I', 'B', 'G', 'K', 'R', 'galdim_angle', 'pmdec', 'pmra', 'galdim_minaxis', 'galdim_majaxis', 'plx_value', 'rvz_redshift', 'mesdistance.dist', 'mesdistance.dist_prec', 'mesdistance.mespos', 'mesdistance.minus_err', 'mesdistance.minus_err_prec', 'mesdistance.plus_err', 'mesdistance.plus_err_prec', 'mesdistance.qual']

✅ Data has been scaled successfully.
Shape of the scaled data: (100000, 27)

First 5 rows of scaled data:


,coo_err_maj,coo_err_min,coo_err_angle,J,V,U,H,I,B,G,...,plx_value,rvz_redshift,mesdistance.dist,mesdistance.dist_prec,mesdistance.mespos,mesdistance.minus_err,mesdistance.minus_err_prec,mesdistance.plus_err,mesdistance.plus_err_prec,mesdistance.qual
0,-0.020655,-0.018584,0.315542,0.368480,0.382774,-0.073609,0.461805,-0.50877,0.713261,-0.333633,...,0.304559,-0.122792,-0.114545,1.397123,0.014233,-0.036721,-0.017900,0.031085,-0.017827,0.0
1,-0.019362,-0.017202,0.315542,-1.235007,-1.131451,-0.073609,-1.198085,-0.50877,-0.958524,1.390686,...,-0.509668,-0.122962,-0.534539,-1.037165,-0.802820,0.060377,-0.023789,-0.066137,-0.023716,0.0
2,-0.018842,-0.016865,0.315542,-1.235007,-1.131451,-0.073609,-1.198085,-0.50877,-0.958524,1.376373,...,-0.509668,-0.122962,-0.534539,-1.037165,-0.802820,0.060377,-0.023789,-0.066137,-0.023716,0.0
3,-0.020769,-0.018730,0.315542,-1.235007,-1.131451,-0.073609,-1.198085,-0.50877,-0.958524,0.977011,...,-0.403682,-0.122962,-0.534539,-1.037165,-0.802820,0.060377,-0.023789,-0.066137,-0.023716,0.0
4,-0.020842,-0.018792,0.315542,-1.235007,-1.131451,-0.073609,-1.198085,-0.50877,-0.958524,0.348612,...,2.673702,-0.122564,-0.427115,1.397123,0.014233,0.059987,-0.017900,-0.065747,-0.017827,0.0


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# --- 6. Build the Autoencoder Model ---

# Define the number of input features
input_dim = X_scaled.shape[1] # This will be 27

# Define the size of the compressed representation (the bottleneck)
# A smaller bottleneck forces the model to learn the most important patterns.
encoding_dim = 8

# --- Define the Encoder ---
# This is the input layer
input_layer = Input(shape=(input_dim,))
# This is the first "encoding" layer
encoder = Dense(16, activation='relu')(input_layer)
# This is the bottleneck layer
encoder = Dense(encoding_dim, activation='relu')(encoder)

# --- Define the Decoder ---
# This layer starts the reconstruction from the bottleneck
decoder = Dense(16, activation='relu')(encoder)
# This is the output layer, which reconstructs the original data
decoder = Dense(input_dim, activation='sigmoid')(decoder) # Sigmoid is good for scaled output

# --- Combine Encoder and Decoder into the Autoencoder Model ---
autoencoder = Model(inputs=input_layer, outputs=decoder)

# --- 7. Compile the Model ---
# We need to tell the model how to learn.
# Optimizer: 'adam' is a very effective and popular choice.
# Loss function: 'mean_squared_error' (mse) measures how different the
# reconstructed data is from the original. The model's goal is to minimize this.
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Print a summary of the model's architecture
print("Model Summary:")
autoencoder.summary()


# --- 8. Train the Model ---
# Now we fit the model to our scaled data.
# epochs: How many times the model will see the entire dataset.
# batch_size: How many samples to process at once.
# validation_split: We'll hold out 20% of the data to check the model's
# performance on data it hasn't seen during training.
print("\nStarting model training...")
history = autoencoder.fit(
    X_scaled, X_scaled, # Note: For autoencoders, input and target are the same
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_split=0.2
)
print("✅ Model training completed!")

Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 27)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 27)             │           459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,187 (4.64 KB)

 Trainable params: 1,187 (4.64 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4193 - val_loss: 0.7150
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7857 - val_loss: 0.6494
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7769 - val_loss: 0.6404
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8010 - val_loss: 0.6369
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7404 - val_loss: 0.6346
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2152 - val_loss: 0.6326
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8504 - val_loss: 0.6311
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.9290 - val_loss: 0.6298
Epoch 9/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8711 - val_loss: 0.6280
Epoch 10/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7731 - val_loss: 0.6268
Epoch 11/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7189 - val_loss: 0.6258
Epoch 12/50
313/313 ━━━━

In [4]:
import numpy as np

# --- 9. Use the Model to Find Anomalies ---

# Get the model's reconstructions of the input data
X_reconstructed = autoencoder.predict(X_scaled)

# Calculate the Mean Squared Error (MSE) for each object.
# This is our "anomaly score". A higher score means more anomalous.
# We calculate the squared difference between the original and reconstructed
# data for each feature, and then take the average across all features for each object.
reconstruction_error = np.mean(np.square(X_scaled - X_reconstructed), axis=1)

print("✅ Calculated reconstruction error for all objects.")

# --- 10. Identify the Most Anomalous Objects ---

# Add the anomaly scores back to our original dataframe
df['anomaly_score'] = reconstruction_error

# Sort the dataframe by the anomaly score in descending order
# to see the most anomalous objects at the top.
df_sorted = df.sort_values(by='anomaly_score', ascending=False)

print("\n🏆 Top 10 Most Anomalous Objects Found:")

# Display the top 10 anomalies, showing their ID, type, and anomaly score.
# We use .reset_index() because the original index is kept after sorting.
display(df_sorted[['main_id', 'otype', 'anomaly_score']].head(10).reset_index(drop=True))

# --- 11. Analyze a Single Anomaly ---
# Let's take a closer look at the single most anomalous object.
most_anomalous_object = df_sorted.iloc[0]

print("\n🔬 In-depth look at the #1 most anomalous object:")
display(most_anomalous_object)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 794us/step
✅ Calculated reconstruction error for all objects.

🏆 Top 10 Most Anomalous Objects Found:


,main_id,otype,anomaly_score
0,GALEX J031858.3+002325,WD*,7524.603144
1,NGC 1545,OpC,1399.080160
2,NGC 1545,OpC,1398.887253
3,NGC 1545,OpC,1398.828569
4,NGC 1545,OpC,1230.558538
5,NGC 1545,OpC,1230.259378
6,NAME X Galaxy in Fornax Cluster,G,992.998213
7,GH 5-1,G,992.950254
8,GH 5-4,G,992.950254
9,GH 7-26,G,992.943970



🔬 In-depth look at the #1 most anomalous object:


main_id                       GALEX J031858.3+002325
otype                                            WD*
sp_type                                         DA1:
rvz_nature                                         s
coo_wavelength                                     O
coo_bibcode                      2020yCat.1350....0G
mesdistance.bibcode              2017MNRAS.469.2102A
mesdistance.method                          ST-L    
mesdistance.unit                                pc  
ra                                         49.742869
dec                                         0.390462
coo_err_maj                                   0.1429
coo_err_min                                   0.1417
coo_err_angle                                   90.0
J                                               19.6
V                                                0.0
U                                                0.0
H                                                0.0
I                                             

In [5]:
import pickle

# --- 12. Save the Model and Scaler for Later Use ---

# Save the trained TensorFlow/Keras model
autoencoder.save('cosmic_curator_model.keras')
print("✅ Autoencoder model saved as 'cosmic_curator_model.keras'")

# Save the StandardScaler object using pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✅ Scaler object saved as 'scaler.pkl'")

✅ Autoencoder model saved as 'cosmic_curator_model.keras'
✅ Scaler object saved as 'scaler.pkl'
